# Augmentation des images des classes minoritaires 

<div>
<b>Objectif:</b>
 Nous avons remarqué un desequilibre de nos quatres classes avec deux minoritaire et deux majoritaire (ce referer a la Data Visualisation de notre jeu de donnée) ce qui induisait des problèmes dans l'entrainement des modèle et donc pour les étapes suivante (F1_score mediocre). L'augmentation d'image à donc été choisi comme outil pour palier au manque de données dans les classes minoritaire et pour réequilibrer les classes et donc le jeu de données
</div>

*Jeu de données* : [Accessible sur Kaggle](https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database)

Après construction de la stratégie générale permettant de resoudre ce déséquilibre, plusieurs combinaisons de paramètres ont été tester afin de determiner celle permettant l'optention des meilleurs metriques (nottament accuracy et F1-score).

A noté que la determination des valeurs des paramètres pour l'optimisation est arbitraire et est vouée a variée en fonction des autres étapes de preprocessing

Pour determiner les scores des différentes preprocessing et donc la pertinance de ces derniers dans le cadre de notre jeu de donnée un modèle `Benchmark` à été développer. Il est important de noter que celui-ci va évoluée et est en aucun cas le modèle final.


**Modèle Benchmark :**
``` python

# Charger MobileNetV2 sans la dernière couche (include_top=False)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False  # On ne touche pas aux poids de MobileNet

# Ajouter notre classifieur (4 classes ici)
inputs = Input(shape=(256, 256, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(4, activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=5)
```
Celui ci à été appliquer à toutes les combinaisons de paramètres comme indiquer dans le tableau suivant :

| ID | Nombre d'occurances | Nombre de couches |Valeur en argument |Bitch size |
|:--------:|:--------:|:--------:|:--------:|:--------:|
|1|4000 |3|0.1|32|
|2|5000|3|0.1|32|
|**3**|**6000**|**3**|**0.1**|**32** |
|4|6000|2|0.1|32|
|5|6000|3 |0.1|32|
|6|6000|4|0.1|32|
|7|6000|3|0.1|32|
|8|6000|3|0.2|32|
|9|6000|3|0.3|32|
|10|6000|3|0.1|16|
|11|6000|3|0.1|32|
|12|6000|3|0.1|64|

<div>
<b>Table 1:</b>
Combinaison de paramètres tester pour leur optimisation
</div>

> La combinaison de paramètre selectionnée correspond à l'`ID 3`, avec un accuracy et un F1-score de 0.92. 

<div>
<b>Pour allez plus loin:</b>
Le test de l'optimisation du modèle benchmark a aussi été réalisé sur les images avec masques en suivant le protocole optimisé et les valeurs de accuracy et de F1-score étaient de 0.89
</div>

# Augmentation d'image après optimisation des paramètres

## Chargement des packages :

In [1]:
# Importation des données 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import tensorflow as tf
import pathlib

#Augmentation des images 
import random
from tensorflow.keras.layers import RandomZoom, RandomRotation, RandomContrast, Rescaling, Resizing, RandomBrightness

#Modelisation
from collections import defaultdict
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras import Input
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import seaborn as sns

## Chargement des différentes classes en local à partir de dossiers : 

### Pour la classe Normale : 

In [2]:
# Chargement des images Normal
folder_path = r"C:\Users\grego\OneDrive\Bureau\Data_Science\Projet\COVID-19_Radiography_Dataset\COVID-19_Radiography_Dataset\Normal\images"
target_size = (256, 256)

# Charger et convertir les images en tableau numpy
images_normal_np = np.array([
    img_to_array(load_img(os.path.join(folder_path, f), target_size=target_size))
    for f in os.listdir(folder_path)
    if f.lower().endswith(('.png', '.jpg', '.jpeg'))
])

print(f"{len(images_normal_np)} images chargées avec la forme {images_normal_np.shape}")

# Créer un Dataset TensorFlow avec les images et les labels (0 pour Normal)
dataset_normal = tf.data.Dataset.from_tensor_slices(
    (images_normal_np, tf.zeros(len(images_normal_np), dtype=tf.int32))
)

10192 images chargées avec la forme (10192, 256, 256, 3)


### Pour la classe Covid : 

In [3]:
# Chargement des images COVID
folder_path = r"C:\Users\grego\OneDrive\Bureau\Data_Science\Projet\COVID-19_Radiography_Dataset\COVID-19_Radiography_Dataset\COVID\images"
target_size = (256, 256)

# Charger et convertir les images en tableau numpy
images_covid_np = np.array([
    img_to_array(load_img(os.path.join(folder_path, f), target_size=target_size))
    for f in os.listdir(folder_path)
    if f.lower().endswith(('.png', '.jpg', '.jpeg'))
])

print(f"{len(images_covid_np)} images chargées avec la forme {images_covid_np.shape}")

# Créer un Dataset TensorFlow avec les images et les labels (1 pour COVID)
dataset_covid = tf.data.Dataset.from_tensor_slices(
    (images_covid_np, tf.ones(len(images_covid_np), dtype=tf.int32))
)

3616 images chargées avec la forme (3616, 256, 256, 3)


### Pour la classe Lung Opacity : 

In [4]:
# Chargement des images Lung Opacity
folder_path = r"C:\Users\grego\OneDrive\Bureau\Data_Science\Projet\COVID-19_Radiography_Dataset\COVID-19_Radiography_Dataset\Lung_opacity\images"
target_size = (256, 256)

# Charger et convertir les images en tableau numpy
images_opacity_np = np.array([
    img_to_array(load_img(os.path.join(folder_path, f), target_size=target_size))
    for f in os.listdir(folder_path)
    if f.lower().endswith(('.png', '.jpg', '.jpeg'))
])

print(f"{len(images_opacity_np)} images chargées avec la forme {images_opacity_np.shape}")

# Créer un Dataset TensorFlow avec les images et les labels (2 pour Lung_opacity)
dataset_opacity = tf.data.Dataset.from_tensor_slices(
    (images_opacity_np, tf.fill(len(images_opacity_np), tf.constant(2, dtype=tf.int32))
))

6012 images chargées avec la forme (6012, 256, 256, 3)


### Pour la classe Viral Pneumonia : 

In [5]:
# Chargement des images Viral Pneumonia
folder_path = r"C:\Users\grego\OneDrive\Bureau\Data_Science\Projet\COVID-19_Radiography_Dataset\COVID-19_Radiography_Dataset\Viral_Pneumonia\images"
target_size = (256, 256)

# Charger et convertir les images en tableau numpy
images_pneumonia_np = np.array([
    img_to_array(load_img(os.path.join(folder_path, f), target_size=target_size))
    for f in os.listdir(folder_path)
    if f.lower().endswith(('.png', '.jpg', '.jpeg'))
])

print(f"{len(images_pneumonia_np)} images chargées avec la forme {images_pneumonia_np.shape}")

# Créer un Dataset TensorFlow avec les images et les labels (3 pour Viral Pneumonia)
dataset_pneumonia = tf.data.Dataset.from_tensor_slices(
    (images_pneumonia_np, tf.fill(len(images_pneumonia_np), tf.constant(2, dtype=tf.int32))
))

1345 images chargées avec la forme (1345, 256, 256, 3)


## Preprocessing/Augmentation d'images : 

In [6]:

# ⚙️ Paramètres
target_per_class = 6000
random.seed(42)

# 🔧 Couches d'augmentation
random_zoom = RandomZoom(0.1)
random_rotation = RandomRotation(0.1)
random_contrast = RandomContrast(0.1)
rescale = Rescaling(1./255)
resize = Resizing(256, 256)  # ⬅️ Adapté à MobileNetV2

augmentations = [random_zoom, random_rotation, random_contrast, resize, rescale]

# 🔄 Fonction d'augmentation
def augment_image(image, augmentations):
    image = tf.image.convert_image_dtype(image, tf.float32)
    for aug_fn in augmentations:
        image = aug_fn(image)
    return image


# 📦 Augmentation par classe
def augmenter_dataset_monoclasse(dataset, label, target, augmentations):
    dataset_list = list(dataset)
    images = [img for img, _ in dataset_list]
    labels = [label] * len(images)

    count = len(images)
    print(f"Classe {label} : {count} images")

    if count > target:
        print(f" ✂️ Réduction à {target} images pour la classe {label}")
        images = images[:target]
        labels = [label] * target

    if count < target:
        to_generate = target - count
        print(f" ➕ Génération de {to_generate} images pour la classe {label}")
        for _ in range(to_generate):
            img = images[random.randint(0, count - 1)]
            aug_img = augment_image(img, augmentations)
            images.append(aug_img)
            labels.append(label)

    return tf.data.Dataset.from_tensor_slices((images, labels))

# 🧪 Appliquer l’augmentation individuellement
dataset_covid     = augmenter_dataset_monoclasse(dataset_covid,     label=1, target=target_per_class, augmentations=augmentations)
dataset_pneumonia = augmenter_dataset_monoclasse(dataset_pneumonia, label=3, target=target_per_class, augmentations=augmentations)
dataset_opacity = dataset_opacity.take(6000)
dataset_normal = dataset_normal.take(6000)

# ✅ Dataset complet équilibré
full_dataset = dataset_covid.concatenate(dataset_pneumonia)\
                               .concatenate(dataset_normal)\
                               .concatenate(dataset_opacity)

full_dataset_list = list(full_dataset)
images, labels = zip(*full_dataset_list)



Classe 1 : 3616 images
 ➕ Génération de 2384 images pour la classe 1
Classe 3 : 1345 images
 ➕ Génération de 4655 images pour la classe 3


## Mise en forme pré-entrainement : 

In [7]:

# Assurer que labels sont des ints hashables
clean_labels = [int(label.numpy()) if isinstance(label, tf.Tensor) else int(label) for label in labels]

# Split STRATIFIÉ (permet d'équilibrée train et validation)
images_by_class = defaultdict(list)
for img, label in zip(images, clean_labels):
    images_by_class[label].append(img)

train_images, train_labels = [], []
val_images, val_labels = [], []

for label, imgs in images_by_class.items():
    split_idx = int(0.8 * len(imgs))
    train_images += imgs[:split_idx]
    train_labels += [label] * split_idx
    val_images += imgs[split_idx:]
    val_labels += [label] * (len(imgs) - split_idx)

# Mélange avec sécurité
combined_train = list(zip(train_images, train_labels))
combined_val = list(zip(val_images, val_labels))
random.shuffle(combined_train)
random.shuffle(combined_val)

if combined_train:
    train_images, train_labels = zip(*combined_train)
else:
    train_images, train_labels = [], []

if combined_val:
    val_images, val_labels = zip(*combined_val)
else:
    val_images, val_labels = [], []

# Générateurs robustes
def generator_train():
    for img, label in zip(train_images, train_labels):
        yield tf.convert_to_tensor(img, dtype=tf.float32), tf.convert_to_tensor(label, dtype=tf.int32)

def generator_val():
    for img, label in zip(val_images, val_labels):
        yield tf.convert_to_tensor(img, dtype=tf.float32), tf.convert_to_tensor(label, dtype=tf.int32)

# Définition de l'output signature pour correspondres a nos attentes 
output_signature = (
    tf.TensorSpec(shape=(256, 256, 3), dtype=tf.float32),
    tf.TensorSpec(shape=(), dtype=tf.int32)
)

# Construction des datasets
train_dataset = tf.data.Dataset.from_generator(generator_train, output_signature=output_signature)
val_dataset = tf.data.Dataset.from_generator(generator_val, output_signature=output_signature)

BATCH_SIZE = 32 # paramètre optimiser 
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Vérification de la stratification
for _, labels_batch in train_dataset.take(1):
    print("✅ Labels du batch train : ", labels_batch.numpy())
for _, labels_batch in val_dataset.take(1):
    print("✅ Labels du batch val : ", labels_batch.numpy())


✅ Labels du batch train :  [1 3 1 3 2 3 2 3 2 0 2 2 3 1 3 3 3 0 3 2 3 3 3 0 0 3 1 1 1 3 2 3]
✅ Labels du batch val :  [2 3 3 3 0 1 1 3 1 3 0 3 1 3 0 1 1 0 3 1 3 3 3 2 0 3 0 1 1 2 3 0]


## Creation et instance du modèle benchmark

In [8]:
# Charger MobileNetV2 sans la dernière couche (include_top=False)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False  # On ne touche pas aux poids de MobileNet

# Ajouter notre classifieur (4 classes ici)
inputs = Input(shape=(256, 256, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(4, activation='softmax')(x)

model = Model(inputs, outputs)

## Entrainement du modèle :

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=5)


Epoch 1/5


600/600 [==============================] - 315s 517ms/step - loss: 0.6912 - accuracy: 0.7136 - val_loss: 0.3019 - val_accuracy: 0.8921
Epoch 2/5
600/600 [==============================] - ETA: 0s - loss: 0.5312 - accuracy: 0.7881

# Metriques : 

In [10]:
y_true = []
y_pred = []

for batch_x, batch_y in val_dataset:
    preds = model.predict(batch_x)
    y_pred.extend(np.argmax(preds, axis=1))  # prédictions argmax
    y_true.extend(batch_y.numpy())  # convertir les labels en numpy

# Optionnel : cast en array
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Rapport
print(classification_report(y_true, y_pred))

# Matrice de confusion
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.title('Matrice de confusion')
plt.show()

# F1-scores
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')
print(f"F1 macro     : {f1_macro:.4f}")
print(f"F1 weighted  : {f1_weighted:.4f}")

: 